In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install hopsworks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import xgboost as xgb


In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [ ]:
%cd /content/drive/MyDrive/jim

/content/drive/MyDrive/jim


In [ ]:
from functions import *

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/jim/training_beijing.csv")

In [ ]:
data_encoder(data)

#data.show(5)

<ipython-input-4-7e22ce190bee>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['conditions'][j]=sum


,aqi,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,snow,snowdepth,windspeed,winddir,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,50,-1.014701,-1.420210,-1.152264,-0.932119,-1.163979,-1.042681,-1.099850,0.302846,-0.360043,...,-0.173316,-0.505713,-0.396431,0.006448,0.252878,-0.285593,-0.719758,-0.713271,-0.786348,42
1,29,-0.955699,-0.998075,-1.026822,-1.045431,-0.824158,-0.978674,-0.746760,0.842623,-0.322425,...,-0.173316,-0.505713,0.239384,0.494859,0.886606,-0.821541,-1.131067,-1.132974,-1.155131,42
2,25,-0.837695,-0.966805,-0.915318,-0.898125,-0.701822,-0.889064,-0.778859,0.415299,-0.307682,...,-0.173316,-0.505713,1.369721,0.418454,0.280954,-0.348646,-1.017759,-1.001817,-1.155131,10
3,27,-0.884897,-0.575939,-0.803814,-1.000106,-0.497929,-0.748248,-0.409720,1.105014,0.852736,...,-0.173316,-0.505713,0.310030,0.351721,1.095175,-0.726962,-1.237577,-1.224784,-1.523915,10
4,21,-1.050102,-0.998075,-0.971070,-1.215399,-0.946494,-1.081085,-0.714661,0.790144,-0.029439,...,-0.173316,-0.505713,0.748036,0.635096,0.657982,-0.033382,-1.012093,-0.988701,-0.786348,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,15,-0.424682,-0.044360,-0.176603,-0.365559,-0.022178,-0.236191,0.135965,0.587728,-0.368346,...,-0.173316,-0.505713,2.189216,0.714402,0.938748,0.187302,-1.218314,-1.224784,-1.155131,10
1508,22,-0.837695,-0.482131,-0.622620,-0.796145,-0.280443,-0.530624,-0.201076,1.120008,-0.469002,...,-0.173316,-0.505713,-0.905083,0.318838,0.902650,-0.695436,-1.327090,-1.316594,-1.523915,8
1509,20,-0.825895,-0.388323,-0.636558,-0.841469,-0.239664,-0.594631,-0.441819,0.497765,-0.469002,...,-0.173316,-0.505713,-0.565982,0.207616,0.168648,0.250355,-1.006428,-0.988701,-0.786348,8
1510,32,-0.991100,-0.544669,-0.720186,-0.909457,-0.389186,-0.633036,-0.409720,0.842623,0.049696,...,-0.173316,-0.505713,-0.212751,0.401046,0.794354,0.029671,-1.375813,-1.395288,-1.523915,10


In [ ]:
train_data, test= train_test_split(data, test_size=0.2, random_state=42)
train_data, validation = train_test_split(train_data, test_size=0.125, random_state=42)

In [ ]:
X = data.fillna(0)
y = X.pop("aqi")

In [ ]:
X_train = train_data.fillna(0)
y_train = X_train.pop("aqi")

In [ ]:
X_test = test.fillna(0)
y_test = X_test.pop("aqi")

In [ ]:
X_val = validation.fillna(0)
y_val = X_val.pop("aqi")

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import xgboost as xgb

# Create the parameter grid for the base models
param_grid_svr = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.1, 0.2, 0.3, 0.4],
    'gamma': [0.1, 0.2, 0.3, 0.4,0.5]
}
param_grid_xgb = {
    'max_depth': [3, 4, 5, 6,7,8],
    'learning_rate': [0.1, 0.2, 0.3, 0.4],
    'n_estimators': [100,150, 200, 250,300,350, 400]
}

# Create the parameter grid for the ensemble model
param_grid_ensemble = {
    'cv': [3, 5, 7]
}

# Create the base models
svr = SVR(kernel='rbf')
xgb_model = xgb.XGBRegressor()

# Create the stacking regressor
ensemble_model = StackingRegressor(estimators=[('svr', svr), ('xgb', xgb_model)])

# Create the grid search models
grid_search_svr = GridSearchCV(svr, param_grid_svr, cv=5, verbose=2, n_jobs=-1)
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=5, verbose=2, n_jobs=-1)
grid_search_ensemble = GridSearchCV(ensemble_model, param_grid_ensemble, cv=5, verbose=2, n_jobs=-1)

# Fit the grid search models to the data
grid_search_svr.fit(X_train, y_train)
grid_search_xgb.fit(X_train, y_train)
grid_search_ensemble.fit(X_train, y_train)

# Print the best parameters and best scores for the base models
print(grid_search_svr.best_params_)
print(grid_search_svr.best_score_)
print(grid_search_xgb.best_params_)
print(grid_search_xgb.best_score_)

# Print the best parameters and best score for the ensemble model
print(grid_search_ensemble.best_params_)
print(grid_search_ensemble.best_score_)


Fitting 5 folds for each of 80 candidates, totalling 400 fits
Fitting 5 folds for each of 168 candidates, totalling 840 fits
[03:49:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 3 candidates, totalling 15 fits
[03:50:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:50:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:50:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:50:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
{'C': 10, 'epsilon': 0.4, 'gamma': 0.1}
0.5948056739416693
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
0.6347648724494513
{'cv': 3}
0.6365168134796532


In [ ]:
best_model=grid_search_ensemble

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
y_pred= best_model.predict(X_test)
print(max(y_pred-y_test), min(y_pred-y_test))
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)

15.857655273368898 -27.346360773130144
6.072281355208845


In [ ]:
y_pred= best_model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print(rmse)

5.075316652641064


In [ ]:
y_pred= best_model.predict(X_val)
print((y_pred-y_val))
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(rmse)

544    -9.572237
982    -0.831140
581    -2.726366
398    -1.518149
293     5.670234
          ...   
699     8.402978
60      1.259756
327    -3.863172
1213   -2.196689
685    -3.066012
Name: aqi, Length: 152, dtype: float64
6.4690292466032595


In [ ]:
y_pred= best_model.predict(X)
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
print(rmse)

5.440906604962268


In [ ]:
import hopsworks

project = hopsworks.login() 

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5333


Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'tempmax', 'type': 'float64'},
   {'name': 'tempmin', 'type': 'float64'},
   {'name': 'temp', 'type': 'float64'},
   {'name': 'feelslikemax', 'type': 'float64'},
   {'name': 'feelslikemin', 'type': 'float64'},
   {'name': 'feelslike', 'type': 'float64'},
   {'name': 'dew', 'type': 'float64'},
   {'name': 'humidity', 'type': 'float64'},
   {'name': 'precip', 'type': 'float64'},
   {'name': 'precipprob', 'type': 'float64'},
   {'name': 'precipcover', 'type': 'float64'},
   {'name': 'snow', 'type': 'float64'},
   {'name': 'snowdepth', 'type': 'float64'},
   {'name': 'windspeed', 'type': 'float64'},
   {'name': 'winddir', 'type': 'float64'},
   {'name': 'cloudcover', 'type': 'float64'},
   {'name': 'visibility', 'type': 'float64'},
   {'name': 'solarradiation', 'type': 'float64'},
   {'name': 'solarenergy', 'type': 'float64'},
   {'name': 'uvindex', 'type': 'float64'},
   {'name': 'conditions', 'type': 'int64'}]},
 'output_schema': {'columnar_

In [ ]:
import joblib

joblib.dump(best_model, 'model.pkl')

['model.pkl']

In [ ]:
model = mr.sklearn.create_model(
    name="zurich_aqi_ensemble",
    metrics={"rmse": "29"},
    description="ensembled aqi prediction model.",
    input_example=X.sample().to_numpy(),
    model_schema=model_schema
)

model.save('model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/5333/models/zurich_aqi_ensemble/2


Model(name: 'zurich_aqi_ensemble', version: 2)